In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
np.random.seed(42)

# Train & Test Set

In [2]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Digit Recog./train.csv')
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Digit Recog./test.csv')

# Separate Labels

In [3]:
y_train = df_train['label']
X_train = df_train.drop(labels=['label'], axis=1)
del df_train

# Pixel values Normalization

In [4]:
X_train = X_train/255.0
df_test = df_test/255.0

Simple model by stacking the 4 layers

In [5]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape = (28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)                             
])

In [6]:
predictions = model(X_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.40584373,  0.22369955,  0.41903475, -0.36501247, -0.12511937,
         0.5401423 ,  0.2324685 , -0.33007234,  0.33099544,  0.0624989 ]],
      dtype=float32)

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.12502633, 0.10420708, 0.12668647, 0.05783928, 0.07352027,
        0.14299689, 0.10512489, 0.05989591, 0.11600997, 0.08869287]],
      dtype=float32)

In [8]:
model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
1313/1313 [==============================] - 3s 2ms/step - loss: 0.3373 - accuracy: 0.9008
Epoch 2/10
1313/1313 [==============================] - 2s 2ms/step - loss: 0.1632 - accuracy: 0.9504
Epoch 3/10
1313/1313 [==============================] - 3s 2ms/step - loss: 0.1229 - accuracy: 0.9627
Epoch 4/10
1313/1313 [==============================] - 2s 2ms/step - loss: 0.0987 - accuracy: 0.9700
Epoch 5/10
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0823 - accuracy: 0.9744
Epoch 6/10
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0719 - accuracy: 0.9771
Epoch 7/10
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0626 - accuracy: 0.9795
Epoch 8/10
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0551 - accuracy: 0.9821
Epoch 9/10
1313/1313 [==============================] - 3s 2ms/step - loss: 0.0508 - accuracy: 0.9828
Epoch 10/10
1313/1313 [==============================] - 2s 2ms/step - loss: 0.046

In [10]:
predictions_1 = model.predict(df_test)

predictions_2 = np.argmax(predictions_1, axis = 1)

final_predictions = pd.Series(predictions_2, name = "Label")

In [11]:
submission = pd.concat([pd.Series(range(1, 28001), name = "ImageId"), final_predictions], axis = 1)

submission.to_csv("tensorflow_crash_course_submission.csv", index = False)